# Project NASA T anomaly onto population

## NOTE: one thing i don't like about this is that the basis data will be different for the rest of the indicators, makes life difficult. Much prefer to use a common basis.

In [18]:
import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import rasterio
from rasterio.io import MemoryFile
from rasterio.warp import reproject, Resampling
from affine import Affine

from config import DATA_SRC, POP_DATA_SRC

In [35]:
nasa_giss_anom = DATA_SRC / 'weather' / 'nasa_giss' / 'air.2x2.1200.mon.anom.comb.nc'
# year for population
year = 2015
population_file_path = POP_DATA_SRC / 'nasa_grid' / 'count' / f'population_{year}_quartres.tif'

In [36]:
nasa_giss = xr.open_dataset(str(nasa_giss_anom))
nasa_giss

<xarray.Dataset>
Dimensions:  (lat: 90, lon: 180, time: 1647)
Coordinates:
  * lat      (lat) float32 89.0 87.0 85.0 83.0 81.0 79.0 77.0 75.0 73.0 71.0 ...
  * lon      (lon) float32 1.0 3.0 5.0 7.0 9.0 11.0 13.0 15.0 17.0 19.0 21.0 ...
  * time     (time) datetime64[ns] 1880-01-01 1880-02-01 1880-03-01 ...
Data variables:
    air      (time, lat, lon) float64 nan nan nan nan nan nan nan nan nan ...
Attributes:
    title:          GISS Surface Temperature Analysis (GISTEMP): 1200km smoot...
    comments:       This is the 1200km smoothed combined land,ocean version o...
    platform:       Analysis
    Source:         http://data.giss.nasa.gov/gistemp/ source and http://data...
    Documentation:  http://www.esrl.noaa.gov/psd/data/gridded/data.gistemp.html
    references:     http://www.esrl.noaa.gov/psd/data/gridded/data.gistemp.html
    Conventions:    COARDS
    history:        created at PSD Jun 2009 by CAS. Converted to chunked, def...
    dataset_title:  GISS Surface Temperature 

In [89]:
temperature = nasa_giss.air.sel(time='2005').mean(dim='time').values.astype('float32')

In [90]:
temperature

array([[ 2.71383071,  2.71383071,  2.71383071, ...,  2.71383071,
         2.71383071,  2.71383071],
       [ 2.71383071,  2.71383071,  2.71383071, ...,  2.71383071,
         2.71383071,  2.71383071],
       [ 2.71383071,  2.71383071,  2.71383071, ...,  2.71383071,
         2.71383071,  2.71383071],
       ..., 
       [ 1.28038037,  1.28038037,  1.28038037, ...,  1.28038037,
         1.28038037,  1.28038037],
       [ 1.28038037,  1.28038037,  1.28038037, ...,  1.28038037,
         1.28038037,  1.28038037],
       [ 1.28038037,  1.28038037,  1.28038037, ...,  1.28038037,
         1.28038037,  1.28038037]], dtype=float32)

In [57]:
with rasterio.open(str(population_file_path)) as pop:
    print(pop.meta)
    pop_meta = pop.meta
    pop_trns = pop.transform
    population = pop.read(1)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 10800, 'height': 4350, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.03333333333333356, 0.0, -180.0,
       0.0, -0.03333333333333356, 85.0000000000092)}


In [42]:
common_crs = pop_meta['crs']

In [10]:
print(population.shape)
print(temperature.shape)
print(np.array(population.shape) / np.array(temperature.shape))

(4350, 10800)
(90, 180)
[ 48.33333333  60.        ]


In [91]:
# with rasterio.open(str(DATA_SRC / 'nasa_temp.tif')) as src:
#     arr = src.read(1)
nasa_trns =  Affine(2, 0.0, -179, 0.0, -2, 89)
newarr = np.empty(shape=(round(population.shape[0]),
                         round(population.shape[1])),
                  dtype=temperature.dtype
                 )

reproject(
    temperature, newarr,
    src_transform = nasa_trns,
    dst_transform = pop_trns,
    src_crs = common_crs,
    dst_crs = common_crs,
    resample = Resampling.cubic_spline)

In [69]:
population.shape

(4350, 10800)

In [92]:
indicator = population * newarr

In [75]:
with rasterio.open(str(DATA_SRC / 'nasa_temp.tif'),
                   'w',
                   driver='GTiff',
                   height=temperature.shape[0],
                   width=temperature.shape[1],
                   count=1,
                   dtype=temperature.dtype,
                   crs=common_crs,
                   transform=nasa_trns,
                   compress='lzw') as new_dataset:
    new_dataset.write(temperature, 1)

In [94]:
with rasterio.open(str(DATA_SRC / 'nasa_dt_indicator.tif'),
                   'w',
                   driver='GTiff',
                   height=indicator.shape[0],
                   width=indicator.shape[1],
                   count=1,
                   dtype=indicator.dtype,
                   crs=common_crs,
                   transform=pop_trns,
                   compress='lzw') as new_dataset:
    new_dataset.write(indicator, 1)